In [1]:
# %load boiler_plate.py
# reload modules when they change
%load_ext autoreload
%autoreload 2

# Alow plotting
%matplotlib inline

# Import modules higher in folder hierarchy
import os
import sys


def add_path(path):
    if path not in sys.path:
        sys.path.append(path)


In [2]:
add_path('..')

In [3]:
from pipeline.db import fetch, inner_join, filter_videos

# Load data
## Videos data

In [4]:
dbname='ds-wizards'
user='wizard'
host='192.95.32.117'
password='GaG23jVxZhMnQaU53r8o'

VQUERY = "select post_id, url from videos where status='ok'"

vres = fetch(host, dbname, user, password, VQUERY)
vres = [(post_id.split("_")[1], url) for post_id, url in vres]

## Tags

In [5]:
dbname='ds-content-tags'
user='ds-content-tags'
password='0fXjWl592vNf1gYvIw8w'
host='192.95.32.117'

TQUERY = "select id, tags from videos where tags is not NULL"
TAGS = "select tag_id, name, path from content_tags"

tres = fetch(host, dbname, user, password, TQUERY)

tags = { 
    tag_id: (name, path) for (tag_id, name, path) in fetch(
        host, dbname, user, password, TAGS) 
}

In [6]:
import itertools
from collections import Counter

In [9]:
c = Counter(itertools.chain(*[tags for (_, tags) in tres]))
c_min10 = sorted([(tags[k][0], v  )for k, v in c.items() if v > 10], key = lambda x: x[1], )

In [10]:
Counter([len(path) for name, path in tags.values()])

Counter({2: 6, 3: 4901, 4: 151})

In [11]:
[(name, path) for name, path in tags.values() if len(path) == 2]

[('Family and relationships', ['Interests', 'Family and relationships']),
 ('Entertainment', ['Interests', 'Entertainment']),
 ('Shopping and fashion', ['Interests', 'Shopping and fashion']),
 ('Sports and outdoors', ['Interests', 'Sports and outdoors']),
 ('Technology', ['Interests', 'Technology']),
 ('Fitness and wellness', ['Interests', 'Fitness and wellness'])]

## Join videos with tags

In [12]:
videos = inner_join(tres, vres)
filtered, t2i, i2t = filter_videos(videos, 10)
print("Found %d videos with %d unique tags" % (len(filtered), len(t2i)))

Found 10052 videos with 514 unique tags


In [13]:
len(t2i)

514

In [14]:
v = list(inner_join(tres, vres))

In [16]:
# Take only the most common class from the list

In [17]:
import math

c = Counter([min([t2i.get(t, math.inf) for t in tags]) for (id, tags, url) in v])

In [18]:
len(c)

420

# DbPedia

In [19]:
import requests

from urllib.parse import quote
from xml.etree import ElementTree

In [20]:
DBPEDIA_QUERY = "http://lookup.dbpedia.org/api/search/KeywordSearch?&QueryString=%s"
LABEL_PATH = ".//{http://lookup.dbpedia.org/}Label"

In [49]:
def dbpedia_labels(tag, only_first = True):
    try:
        response = requests.get(DBPEDIA_QUERY % quote(tag))
        tree = ElementTree.fromstring(response.content)
        tree = tree[0] if only_first else tree

        labels = [label.text for label in tree.findall(LABEL_PATH)]
        labels = [label for label in labels if label!= "owl#Thing"]
    except Exception as e:
        return []
    
    return labels

# Explore DbPedia tags

In [50]:
for tag_id, (name, path) in itertools.islice(tags.items(), 5):
    print(tag_id, name)
    for tag in dbpedia_labels(name):
        print("\t %s" % tag)

6003392922131 Dialogue
	 Scots language
	 language
	 language
	 Languages of Ireland
	 English languages
	 Languages of Scotland
	 Scots language
	 Subject–verb–object languages
6003216719230 Raven (drag queen)
	 Raven (drag queen)
	 http://xmlns.com/foaf/0.1/ person
	 person
	 agent
	 person
	 American people of Russian descent
	 Gay actors
	 Living people
	 Drag queens
	 1979 births
	 LGBT people from the United States
	 American performance artists
	 People from Riverside, California
	 American female impersonators
	 Former Latter Day Saints
	 Participants in American reality television series
6003274262708 Fiction books
	 List of science fiction novels
	 Science fiction novels
	 Lists of books by genre
	 Lists of novels
	 Science fiction lists
6003352347000 Scarf
	 Hijab
	 Islam and women
	 Arabic clothing
	 Headgear
	 Purdah
	 Islamic dress
	 Women's rights in religious movements
	 Scarves
	 Islamic dress (female)
6003392101554 Lawyer
	 Lawyer
	 Occupations
	 Legal ethics
	 Lawyer

In [54]:
t2l = {tag_id: dbpedia_labels(name) for tag_id, (name, path) in tags.items()}

In [90]:
def keep(label):
    if "from" in label:
        return False
    
    if "births" in label:
        return False
    
    if "xmlns" in label:
        return False
    
    return True

def normalise(label):
    return label.lower()

In [91]:
labels_raw = itertools.chain(*[labels for (tag_id, labels) in t2l.items()])
labels_raw = [normalise(label) for label in labels_raw if keep(label)]
labels = Counter(labels_raw)

In [93]:
labels.most_common()[:50]

[('person', 1524),
 ('agent', 1230),
 ('living people', 673),
 ('place', 572),
 ('organization', 468),
 ('organisation', 468),
 ('work', 399),
 ('creative work', 399),
 ('populated place', 252),
 ('american film actors', 240),
 ('country', 220),
 ('american television actors', 211),
 ('company', 199),
 ('artist', 185),
 ('music group', 178),
 ('television show', 153),
 ('musical artist', 140),
 ('film', 139),
 ('movie', 135),
 ('city', 135),
 ('english-language television series', 119),
 ('american voice actors', 104),
 ('grammy award winners', 101),
 ('member states of the united nations', 100),
 ('settlement', 93),
 ('english-language films', 93),
 ('television station', 88),
 ('2000s american television series', 80),
 ('2010s american television series', 76),
 ('event', 76),
 ('american child actors', 75),
 ('office holder', 74),
 ('species', 74),
 ('greek loanwords', 72),
 ('eukaryote', 71),
 ('american people of irish descent', 63),
 ('fictional character', 61),
 ('broadcaster', 5